In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install tensorflow

In [ ]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.svm import SVC
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline
# from sklearn.metrics import classification_report, confusion_matrix

# # File paths
# audio_feature_files = [
#     "/content/drive/My Drive/Project/savee_features.csv",
#     "/content/drive/My Drive/Project/tess_features_audio.csv",
#     "/content/drive/My Drive/Project/crema_features_audio.csv"
# ]

# text_feature_files = [
#     "/content/drive/My Drive/Project/SAVEE_feature_vectors_with_labels.csv",
#     "/content/drive/My Drive/Project/tess_features_emotions.csv",
#     "/content/drive/My Drive/Project/crema_features_emotions.csv"
# ]

# # Combine all datasets
# combined_features = []
# file_names = []
# emotions = []

# for audio_file, text_file in zip(audio_feature_files, text_feature_files):
#     # Load audio and text features
#     audio_features = pd.read_csv(audio_file)
#     text_features = pd.read_csv(text_file)

#     # Ensure alignment of files by filename
#     if "File_Name" not in audio_features.columns:
#         audio_features["File_Name"] = text_features["File_Name"]

#     # Check emotion alignment
#     for i in range(len(audio_features)):
#         if audio_features.loc[i, "Emotion"] == text_features.loc[i, "Emotion"]:
#             combined_row = {
#                 "File_Name": text_features.loc[i, "File_Name"],
#                 "Emotion": audio_features.loc[i, "Emotion"],
#             }
#             # Combine audio and text features
#             combined_row.update(audio_features.iloc[i, :].filter(like="Feature_").to_dict())
#             combined_row.update(text_features.iloc[i, :].filter(like="Feature_").to_dict())
#             combined_features.append(combined_row)

# # Create a DataFrame for the combined features
# combined_df = pd.DataFrame(combined_features)
# file_names = combined_df["File_Name"].tolist()
# emotions = combined_df["Emotion"].tolist()

# # Extract feature columns
# feature_columns = [col for col in combined_df.columns if col.startswith("Feature_")]
# X = combined_df[feature_columns].values
# y = emotions

# print(f"Combined feature matrix shape: {X.shape}")
# print(f"Labels shape: {len(y)}")

# # Split data into training and testing sets
# X_train, X_test, y_train, y_test, file_names_train, file_names_test = train_test_split(
#     X, y, file_names, test_size=0.2, random_state=42, stratify=y
# )

# # Create a pipeline with scaling and SVM
# svm_pipeline = Pipeline([
#     ("scaler", StandardScaler()),  # Scale features for better SVM performance
#     ("svm", SVC(kernel="linear", probability=True, random_state=42))  # Linear kernel SVM
# ])

# # Train the SVM model
# svm_pipeline.fit(X_train, y_train)

# # Predict on the test set
# y_pred = svm_pipeline.predict(X_test)

# # Evaluate the model
# print("Classification Report:")
# print(classification_report(y_test, y_pred))

# print("Confusion Matrix:")
# print(confusion_matrix(y_test, y_pred))

# # Create output DataFrame with filenames and predicted labels
# output_df = pd.DataFrame({
#     "File_Name": file_names_test,
#     "True_Emotion": y_test,
#     "Predicted_Emotion": y_pred
# })

# # Save output to CSV
# output_csv_path = "/content/drive/My Drive/Project/predicted_emotions.csv"
# output_df.to_csv(output_csv_path, index=False)
# print(f"Predicted emotions saved to {output_csv_path}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix

# File paths
audio_feature_files = [
    "/content/drive/My Drive/Project/savee_features.csv",
    "/content/drive/My Drive/Project/tess_features_audio.csv",
    "/content/drive/My Drive/Project/crema_features_audio.csv"
]

text_feature_files = [
    "/content/drive/My Drive/Project/SAVEE_feature_vectors_with_labels.csv",
    "/content/drive/My Drive/Project/tess_features_emotions.csv",
    "/content/drive/My Drive/Project/crema_features_emotions.csv"
]

# Combine all datasets
combined_features = []
file_names = []
emotions = []

for audio_file, text_file in zip(audio_feature_files, text_feature_files):
    # Load audio and text features
    audio_features = pd.read_csv(audio_file)
    text_features = pd.read_csv(text_file)

    # Ensure alignment of files by filename
    if "File_Name" not in audio_features.columns:
        audio_features["File_Name"] = text_features["File_Name"]

    # Check emotion alignment
    for i in range(len(audio_features)):
        if audio_features.loc[i, "Emotion"] == text_features.loc[i, "Emotion"]:
            combined_row = {
                "File_Name": text_features.loc[i, "File_Name"],
                "Emotion": audio_features.loc[i, "Emotion"],
            }
            # Combine audio and text features
            combined_row.update(audio_features.iloc[i, :].filter(like="Feature_").to_dict())
            combined_row.update(text_features.iloc[i, :].filter(like="Feature_").to_dict())
            combined_features.append(combined_row)

# Create a DataFrame for the combined features
combined_df = pd.DataFrame(combined_features)
file_names = combined_df["File_Name"].tolist()
emotions = combined_df["Emotion"].tolist()

# Extract feature columns
feature_columns = [col for col in combined_df.columns if col.startswith("Feature_")]
X = combined_df[feature_columns].values
y = emotions

print(f"Combined feature matrix shape: {X.shape}")
print(f"Labels shape: {len(y)}")

# Convert labels to categorical (one-hot encoding)
y = pd.Categorical(y).codes
y = to_categorical(y)  # Convert to one-hot encoded labels

# Split data into training and testing sets
X_train, X_test, y_train, y_test, file_names_train, file_names_test = train_test_split(
    X, y, file_names, test_size=0.2, random_state=42, stratify=y
)

# Reshape the input for CNN (adding an extra dimension)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)  # Adding the channel dimension
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)  # Adding the channel dimension

# Create the CNN model
cnn_model = Sequential()

# Add Conv1D layer (for 1D sequence data like feature vectors)
cnn_model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Dropout(0.2))

# Add more Conv1D layers to build complexity
cnn_model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Dropout(0.2))

# Flatten the output from the convolution layers
cnn_model.add(Flatten())

# Add Dense layers for classification
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(y.shape[1], activation='softmax'))  # Output layer for multi-class classification

# Compile the model
cnn_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the CNN model
cnn_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the test set
y_pred = np.argmax(cnn_model.predict(X_test), axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test_labels, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test_labels, y_pred))

# Create output DataFrame with filenames and predicted labels
output_df = pd.DataFrame({
    "File_Name": file_names_test,
    "True_Emotion": y_test_labels,
    "Predicted_Emotion": y_pred
})

# Save output to CSV
output_csv_path = "/content/drive/My Drive/Project/predicted_emotions_cnn.csv"
output_df.to_csv(output_csv_path, index=False)
print(f"Predicted emotions saved to {output_csv_path}")


Combined feature matrix shape: (10722, 1025)
Labels shape: 10722


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
269/269 ━━━━━━━━━━━━━━━━━━━━ 57s 201ms/step - accuracy: 0.1700 - loss: 1.9110 - val_accuracy: 0.1967 - val_loss: 1.8716
Epoch 2/20
269/269 ━━━━━━━━━━━━━━━━━━━━ 55s 203ms/step - accuracy: 0.1987 - loss: 1.8589 - val_accuracy: 0.2205 - val_loss: 1.8073
Epoch 3/20
269/269 ━━━━━━━━━━━━━━━━━━━━ 80s 196ms/step - accuracy: 0.2082 - loss: 1.8228 - val_accuracy: 0.2214 - val_loss: 1.7933
Epoch 4/20
269/269 ━━━━━━━━━━━━━━━━━━━━ 84s 202ms/step - accuracy: 0.2287 - loss: 1.8013 - val_accuracy: 0.2550 - val_loss: 1.7687
Epoch 5/20
269/269 ━━━━━━━━━━━━━━━━━━━━ 53s 197ms/step - accuracy: 0.2406 - loss: 1.7871 - val_accuracy: 0.2429 - val_loss: 1.7607
Epoch 6/20
269/269 ━━━━━━━━━━━━━━━━━━━━ 52s 192ms/step - accuracy: 0.2427 - loss: 1.7702 - val_accuracy: 0.2853 - val_loss: 1.7471
Epoch 7/20
269/269 ━━━━━━━━━━━━━━━━━━━━ 84s 198ms/step - accuracy: 0.2355 - loss: 1.7773 - val_accuracy: 0.2867 - val_loss: 1.7397
Epoch 8/20
269/269 ━━━━━━━━━━━━━━━━━━━━ 81s 195ms/step - accuracy: 0.2604 - loss: 1

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix

# File paths
audio_feature_files = [
    "/content/drive/My Drive/Project/savee_features.csv",
    "/content/drive/My Drive/Project/tess_features_audio.csv",
    "/content/drive/My Drive/Project/crema_features_audio.csv"
]

text_feature_files = [
    "/content/drive/My Drive/Project/SAVEE_feature_vectors_with_labels.csv",
    "/content/drive/My Drive/Project/tess_features_emotions.csv",
    "/content/drive/My Drive/Project/crema_features_emotions.csv"
]

# Combine all datasets
combined_features = []
file_names = []
emotions = []

for audio_file, text_file in zip(audio_feature_files, text_feature_files):
    # Load audio and text features
    audio_features = pd.read_csv(audio_file)
    text_features = pd.read_csv(text_file)

    # Ensure alignment of files by filename
    if "File_Name" not in audio_features.columns:
        audio_features["File_Name"] = text_features["File_Name"]

    # Check emotion alignment
    for i in range(len(audio_features)):
        if audio_features.loc[i, "Emotion"] == text_features.loc[i, "Emotion"]:
            combined_row = {
                "File_Name": text_features.loc[i, "File_Name"],
                "Emotion": audio_features.loc[i, "Emotion"],
            }
            # Combine audio and text features
            combined_row.update(audio_features.iloc[i, :].filter(like="Feature_").to_dict())
            combined_row.update(text_features.iloc[i, :].filter(like="Feature_").to_dict())
            combined_features.append(combined_row)

# Create a DataFrame for the combined features
combined_df = pd.DataFrame(combined_features)
file_names = combined_df["File_Name"].tolist()
emotions = combined_df["Emotion"].tolist()

# Extract feature columns
feature_columns = [col for col in combined_df.columns if col.startswith("Feature_")]
X = combined_df[feature_columns].values
y = emotions

print(f"Combined feature matrix shape: {X.shape}")
print(f"Labels shape: {len(y)}")

# Convert labels to categorical (one-hot encoding)
y = pd.Categorical(y).codes
y = to_categorical(y)  # Convert to one-hot encoded labels

# Split data into training and testing sets
X_train, X_test, y_train, y_test, file_names_train, file_names_test = train_test_split(
    X, y, file_names, test_size=0.2, random_state=42, stratify=y
)

# Reshape the input for RNN/LSTM (adding an extra dimension for time steps)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)  # Adding time steps (1 time step per feature vector)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)  # Adding time steps (1 time step per feature vector)

# Create the LSTM model
lstm_model = Sequential()

# Add LSTM layer
lstm_model.add(LSTM(128, input_shape=(X_train.shape[1], 1), activation='tanh', return_sequences=False))
lstm_model.add(Dropout(0.2))

# Add Dense layer for classification
lstm_model.add(Dense(128, activation='relu'))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(y.shape[1], activation='softmax'))  # Output layer for multi-class classification

# Compile the model
lstm_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the LSTM model
lstm_model.fit(X_train, y_train, epochs=2, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model on the test set
y_pred = np.argmax(lstm_model.predict(X_test), axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test_labels, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test_labels, y_pred))

# Create output DataFrame with filenames and predicted labels
output_df = pd.DataFrame({
    "File_Name": file_names_test,
    "True_Emotion": y_test_labels,
    "Predicted_Emotion": y_pred
})

# Save output to CSV
output_csv_path = "/content/drive/My Drive/Project/predicted_emotions_lstm.csv"
output_df.to_csv(output_csv_path, index=False)
print(f"Predicted emotions saved to {output_csv_path}")


Combined feature matrix shape: (10722, 1025)
Labels shape: 10722
Epoch 1/2


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


135/135 ━━━━━━━━━━━━━━━━━━━━ 256s 2s/step - accuracy: 0.1519 - loss: 1.9146 - val_accuracy: 0.1608 - val_loss: 1.8812
Epoch 2/2
135/135 ━━━━━━━━━━━━━━━━━━━━ 287s 2s/step - accuracy: 0.1811 - loss: 1.8891 - val_accuracy: 0.1828 - val_loss: 1.8684
68/68 ━━━━━━━━━━━━━━━━━━━━ 35s 521ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       346
           1       0.16      0.56      0.25       347
           2       0.00      0.00      0.00       346
           3       0.00      0.00      0.00       346
           4       0.00      0.00      0.00       322
           5       0.21      0.58      0.31       346
           6       0.00      0.00      0.00        92

    accuracy                           0.18      2145
   macro avg       0.05      0.16      0.08      2145
weighted avg       0.06      0.18      0.09      2145

Confusion Matrix:
[[  0 268   0   0   0  78   0]
 [  0 193   0   0   0 154   0]
 [  0 157   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Assuming X is your feature matrix and y is the target labels

# Check if `y` is one-hot encoded by printing its shape
print(f"Shape of y before processing: {y.shape}")

# If y is one-hot encoded, convert it back to integer labels using np.argmax
if len(y.shape) > 1 and y.shape[1] > 1:
    y = np.argmax(y, axis=1)  # Convert one-hot encoding to integer labels
    print("y is one-hot encoded, converted to integer labels.")

print(f"Shape of y after processing: {y.shape}")

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Create RandomForest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Shape of y before processing: (10722, 7)
y is one-hot encoded, converted to integer labels.
Shape of y after processing: (10722,)
Classification Report:
              precision    recall  f1-score   support

           0       0.44      0.49      0.47       346
           1       0.41      0.40      0.40       346
           2       0.42      0.38      0.40       346
           3       0.32      0.31      0.32       347
           4       0.35      0.38      0.37       322
           5       0.42      0.43      0.43       346
           6       0.68      0.49      0.57        92

    accuracy                           0.40      2145
   macro avg       0.43      0.41      0.42      2145
weighted avg       0.41      0.40      0.40      2145

Confusion Matrix:
[[170  31  38  41  53  11   2]
 [ 41 138  36  34  20  74   3]
 [ 33  33 132  63  26  55   4]
 [ 47  33  59 109  66  27   6]
 [ 67  20  15  60 122  35   3]
 [ 22  66  35  25  46 149   3]
 [  4  15   1  12  12   3  45]]


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Convert labels to integers if they are categorical
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Create MLP model
mlp_model = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, activation='relu', solver='adam', random_state=42)

# Train the model
mlp_model.fit(X_train, y_train)

# Predict on the test set
y_pred = mlp_model.predict(X_test)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.62      0.59       346
           1       0.63      0.59      0.61       346
           2       0.47      0.59      0.53       346
           3       0.42      0.47      0.44       347
           4       0.62      0.50      0.56       322
           5       0.64      0.49      0.55       346
           6       0.70      0.77      0.74        92

    accuracy                           0.55      2145
   macro avg       0.58      0.58      0.57      2145
weighted avg       0.56      0.55      0.55      2145

Confusion Matrix:
[[213  18  25  58  25   5   2]
 [ 38 204  34  29   8  30   3]
 [ 20  12 205  64   7  31   7]
 [ 45  23  72 164  21   7  15]
 [ 37  12  39  49 162  21   2]
 [ 14  47  55  24  37 168   1]
 [  5   7   2   6   0   1  71]]
